# Train Cogames

this notebook will help you to get started with cogames with the simplest training example.

Make sure you have selected T4 runtime!

## Install Dependencies

In [5]:
!git clone --filter=blob:none https://github.com/Metta-AI/metta.git
%cd metta

fatal: destination path 'metta' already exists and is not an empty directory.
/content/metta/metta


### Install Bazel

In [ ]:
!sudo apt install apt-transport-https curl gnupg -y
!curl -fsSL https://bazel.build/bazel-release.pub.gpg | sudo apt-key add -
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!sudo apt update && sudo apt install bazel -y
!bazel --version


### Install Nim

In [ ]:
!sudo apt-get update -y
!sudo apt-get install -y curl git build-essential
!curl https://nim-lang.org/choosenim/init.sh -sSf | sh -s -- -y
import os
os.environ["PATH"] += ":/root/.nimble/bin"


!nim --version
!nimble --version


### Install all the required packages

In [ ]:
!uv run cogames

## Training Code + PPO Loss

It includes a complete reinforcement learning training loop that implements a rollout buffer for generalized advantage estimation (GAE), a PPO update function with clipping, entropy regularization, and value loss computation, and supports both simple and LSTM-based policies. It interacts with the MettaGridEnv simulation, collecting observations, actions, and rewards over multiple epochs to optimize the policy network.

You can modify mission configurations, hyperparameters, and policy types to experiment with different training setups.

In [ ]:
!uv run cogames train --mission training_facility.harvest --variant lonely_heart --variant heart_chorus --variant pack_rat --variant extractor_base